In [48]:
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn, rnn
from sklearn import metrics
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn
import pickle
import random
import time
from tqdm import tqdm, tqdm_notebook
import sys
sys.path.append('/data/CaoZhong/utils/')
from my_utils import *

# 加载数据

In [2]:
ctx = mx.gpu(0)
train_batch_size = 64
test_batch_size = 128
model_name = 'din_basemodel_gluon'
with open('/data/CaoZhong/data/dien/dataset_sub_gluon.pkl', 'rb') as f:
    train_set = pickle.load(f)
    test_set = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)
random.shuffle(train_set)
cate_list = nd.array(cate_list)
print("user count: %d\titem count: %d\tcate count: %d" % (user_count, item_count, cate_count))
print("train set len: ",len(train_set))
print('test set len: ', len(test_set))
train_set = train_set[:10000]
test_set = test_set[:1000]

user count: 1053	item count: 63001	cate count: 801
train set len:  129888
test set len:  2106


In [3]:
def get_data(data_set):    
    all_user, all_hist,all_hist_neg, all_pre,  all_label = [],[],[],[],[]
    for u, hist,hist_neg, pre, label in data_set:
        all_user.append(u)
        all_hist.append(hist)
        all_hist_neg.append(hist_neg)
        all_pre.append(pre)
        all_label.append(label)
    return all_user, all_hist, all_hist_neg, all_pre, all_label

In [4]:
def batchify(data):
    max_len = max([len(h) for u, h, h_neg, p, l in data])
    uid, hist,hist_neg, pre, label, sl= [], [], [], [], [],[]
    for u, h, h_neg, p, l in data:
        uid += [u]
        sl += [len(h)]
        pre += [p]
        hist += [h + [0] * (max_len-len(h))]
        hist_neg += [h_neg + [[0]*5]*(max_len - len(h))]
        label += [l]
    uid = nd.array(uid).reshape((-1))
    hist_item = nd.array(hist)
    hist_cate = cate_list[hist_item]
    hist_item_neg = nd.array(hist_neg)
    hist_cate_neg = cate_list[hist_item_neg]
    item = nd.array(pre).reshape((-1))
    cate = cate_list[item]
    label = nd.array(label).reshape((-1))
    seq_len = nd.array(sl).reshape((-1))
    
    return (uid, hist_item, hist_cate, hist_item_neg, hist_cate_neg, item, cate, label, seq_len, max_len)

In [5]:
# 训练集
all_user, all_hist, all_hist_neg, all_pre, all_label = get_data(train_set)
dataset = gdata.ArrayDataset(all_user, all_hist, all_hist_neg, all_pre, all_label)
# 测试集
all_user, all_hist, all_hist_neg, all_pre, all_label = get_data(test_set)
dataset_test = gdata.ArrayDataset(all_user, all_hist, all_hist_neg, all_pre, all_label)

# 建立数据迭代器
train_iter = gdata.DataLoader(dataset, train_batch_size, shuffle=True, batchify_fn=batchify)
test_iter = gdata.DataLoader(dataset_test, test_batch_size, shuffle=True, batchify_fn=batchify)

In [6]:
for batch in train_iter:
    for name, data in zip(['uid','hist','hist_cate','hist_item_neg','hist_cate_neg','pre','pre_cate','label','sl'], batch):
        print(name, 'shape: ', data.shape)    
    print(batch[-1])
    break
for batch in test_iter:
    for name, data in zip(['uid','hist','hist_cate','hist_item_neg','hist_cate_neg','pre','pre_cate','label','sl'], batch):
        print(name, 'shape: ', data.shape)        
    break

uid shape:  (64,)
hist shape:  (64, 281)
hist_cate shape:  (64, 281)
hist_item_neg shape:  (64, 281, 5)
hist_cate_neg shape:  (64, 281, 5)
pre shape:  (64,)
pre_cate shape:  (64,)
label shape:  (64,)
sl shape:  (64,)
281
uid shape:  (128,)
hist shape:  (128, 430)
hist_cate shape:  (128, 430)
hist_item_neg shape:  (128, 430, 5)
hist_cate_neg shape:  (128, 430, 5)
pre shape:  (128,)
pre_cate shape:  (128,)
label shape:  (128,)
sl shape:  (128,)


# 建立模型

In [7]:
"""
function: 辅助函数损失
Parameters: outpus - [B, T-1, 2E]
            hist_item - [B, T-1, 2E]
            noclk_hist_item - [B, T-1, 2E]
            seq_len - [B,]
Returns: 
"""
def auxiliary_loss(net, h_states, hist_item, noclk_hist_item,seq_len):
    print(h_states.shape, hist_item.shape, noclk_hist_item.shape,seq_len.shape)
#     click_input = nd.concat(h_states, hist_item,dim=-1)
#     noclick_input = nd.concat(h_states, noclk_hist_item, dim=-1)
#     click_prop = net(click_input)
#     click_prop = click_prop.reshape((-1, h_states.shape[1]))
#     noclick_prop = net(noclick_input)
#     noclick_prop = noclick_prop.reshape((-1, h_states.shape[1]))
#     click_loss = - nd.log(click_prop)
#     noclick_loss = -nd.log(1 - noclick_prop)
#     print(click_loss.shape,noclick_loss.shape)
#     loss_ = (click_loss + noclick_loss).mean(axis=1)
    a = nd.concat(h_states, hist_item, noclk_hist_item)
    b = net(a)
    return b

In [8]:
class Attention(nn.Block):
    def __init__(self, attention_size, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.model = nn.Sequential()
        self.model.add(nn.Dense(attention_size, activation='tanh', use_bias=False, flatten=False))
        self.model.add(nn.Dense(1, use_bias=False, flatten=False))
        
    """
    Parameters:
        query:    [B, H]
        keys:     [T, B, H]
        sl:       [B]
    """
    def forward(self, keys, query,sl):
        query = nd.broadcast_axis(query.expand_dims(0), axis=0, size=keys.shape[0])
        item_history = nd.concat(keys, query, dim=2)
        e = self.model(item_history)   # [T, B, 1]
        e = nd.SequenceMask(e, sl.reshape((-1)), use_sequence_length=True, value=(-2 ** 32 + 1))
        alpha = nd.softmax(e, axis=0)        # [T, B, 1]

        return (alpha * keys).sum(axis=0)  # [T, B, 1] * [T, B, H]

In [59]:
class Model(nn.Block):
    def __init__(self, n_uid, n_mid, n_cat, embed_size, hidden_size, attention_size, ctx, **kwargs):
        super(Model, self).__init__(**kwargs)
        self.hidden_size = hidden_size
        self.attention_size = attention_size
        self.embed_size = embed_size
        self.uid_embedding = nn.Embedding(n_uid, 2*embed_size)
        self.item_embedding = nn.Embedding(n_mid, embed_size)
        self.cate_embedding = nn.Embedding(n_cat, embed_size)
        self.rnn1 = rnn.GRU(hidden_size)
        self.rnn2 = rnn.GRU(hidden_size)
        self.attention_layer = Attention(attention_size)
        
#         self.aux_net = nn.Sequential()
#         self.aux_net.add(nn.BatchNorm())
#         self.aux_net.add(nn.Dense(100, activation='sigmoid', flatten=False))
#         self.aux_net.add(nn.Dense(50, activation='sigmoid', flatten=False))
#         self.aux_net.add(nn.Dense(1, activation='sigmoid', flatten=False))
        
        
        self.mlp = nn.Sequential()
        self.mlp.add(nn.BatchNorm())
        self.mlp.add(nn.Dense(200, activation='relu'))
        self.mlp.add(nn.Dense(80, activation='relu'))
        self.mlp.add(nn.Dense(1, activation=None))
    
    def aux_loss(self, h_states, hist_item, noclk_hist_item,seq_len):
#         seq_len = seq_len - 1 
#         click_input = nd.concat(h_states, hist_item,dim=-1)
#         noclick_input = nd.concat(h_states, noclk_hist_item, dim=-1)
#         click_prop = self.aux_net(click_input)
#         click_prop = click_prop.reshape((h_states.shape[0], h_states.shape[1]))
#         noclick_prop = self.aux_net(noclick_input)
#         noclick_prop = noclick_prop.reshape((h_states.shape[0], h_states.shape[1]))  # [B, T-1]
#         click_loss = - nd.log(click_prop)
#         noclick_loss = -nd.log(1 - noclick_prop)
#         loss_ = (click_loss + noclick_loss).mean(axis=1)   # [B]
        loss_ = self.aux_net(h_states).sum(axis=-1)
        loss_ = loss_.mean(axis=-1)
        return loss_
    
    def forward(self, uid, hist_item, hist_cate, noclk_hist_item, noclk_hist_cate,item, cate, seq_len):
        uid_embed = self.uid_embedding(uid)
        item_idx_embed = self.item_embedding(item)
        cate_idx_embed = self.cate_embedding(cate)
        item_embed = nd.concat(item_idx_embed, cate_idx_embed, dim=-1)
        
        hist_item_idx_embed = self.item_embedding(hist_item)
        hist_cate_idx_embed = self.cate_embedding(hist_cate)
        hist_item_embed = nd.concat(hist_item_idx_embed, hist_cate_idx_embed, dim=-1)  # [B, T, 2E]
        hist_item_sum = nd.SequenceMask(hist_item_embed.swapaxes(0, 1), sequence_length=seq_len.reshape((-1)), use_sequence_length=True) # [T, B, H]
        hist_item_sum = nd.sum(hist_item_sum, axis=0)  # [B, 2E]
        
        noclk_hist_item = self.item_embedding(noclk_hist_item)
        noclk_hist_cate = self.item_embedding(noclk_hist_cate)
        noclk_hist = nd.concat(noclk_hist_item, noclk_hist_cate, dim=-1)
        noclk_hist_item_embed = noclk_hist[:,:,0,:]
        noclk_hist_item_embed = noclk_hist_item_embed.reshape((-1, hist_item_embed.shape[1],hist_item_embed.shape[-1]))  # [B, T, 2E]

        rnn_outputs = self.rnn1(hist_item_embed.swapaxes(0,1))  # [T, B, H]
        
        rnn_out2 = self.rnn2(rnn_outputs)
        c = self.attention_layer(rnn_out2, item_embed, seq_len)
        
#         aux_loss = self.aux_loss(rnn_outputs.swapaxes(0,1)[:,:-1,:], hist_item_embed[:,1:,:], noclk_hist_item_embed[:,1:,:], seq_len)

        inp = nd.concat(uid_embed, item_embed,hist_item_sum, item_embed*hist_item_sum, c, dim=1)
        
        
        score = self.mlp(inp)
        score = score.reshape((-1))
        return score
            

# 模型训练

In [60]:
def eval_auc(net,data_iter, ctx):
    score = None
    y = None
    for batch in data_iter:
        uid, hist_item, hist_cate, hist_item_neg, hist_cate_neg,item,cate, label, sl = [data.as_in_context(ctx) for data in batch[:-1]]
        if score is None:
            out = net(uid, hist_item, hist_cate,  hist_item_neg, hist_cate_neg,item, cate, sl)
            score = nd.sigmoid(out)
            y = label
        else:
            out = net(uid, hist_item, hist_cate,  hist_item_neg, hist_cate_neg,item, cate, sl)
            score = nd.concat(score, nd.sigmoid(out), dim=0)
            y = nd.concat(y, label, dim=0)
    y = list(y.asnumpy())
    score = list(score.asnumpy())
    fpr,tpr,thresholds = metrics.roc_curve(y,score)
    auc = metrics.auc(fpr,tpr)
    return auc

In [65]:
def train(net, train_iter, test_iter,loss,train_batch_size,lr, num_epochs, ctx,log_dir,loss_name='loss',auc_name='auc'):
    auc_list, loss_list = [], []
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':lr})
    
    global_step = 1
    stime = time.time()
    stime2 = time.time()
    
    loss_val, auc_val, time_val = 0.0, 0.0, 0.0
    print('auc: %.4f' % (eval_auc(net,test_iter, ctx)))
    epoch_bar = tqdm_notebook(range(1, num_epochs+1))
    for epoch in epoch_bar:
        l_sum = 0.0
        bar = tqdm_notebook(train_iter)
        for batch in bar:
            uid, hist_item, hist_cate, hist_item_neg, hist_cate_neg,item,cate, label, sl = [data.as_in_context(ctx) for data in batch[:-1]]
            with autograd.record():
                pred = net(uid, hist_item, hist_cate,  hist_item_neg, hist_cate_neg,item, cate, sl)
                l = loss(pred, label)
#                 cross_loss = loss(pred, label)
#                 print(aux_loss.shape,cross_loss.shape)
#                 l = aux_loss + cross_loss
            l.backward()
            trainer.step(train_batch_size)
            l_sum += l.mean().asscalar()            
            if global_step % 200 ==0:
                test_auc = eval_auc(net, test_iter, ctx)
                loss_val = l_sum / 200
                auc_val = test_auc
                time_val = time.time()-stime2
                tip = "epoch %d, global step:%d, loss %.4f, test auc:%.4f, time:%.2f" % (epoch,global_step, loss_val, auc_val, time_val )
                bar.set_description_str(tip)
                tip_info(tip,out=False)
                loss_list.append(loss_val)
                auc_list.append(auc_val)
                l_sum = 0.0
                stime2 = time.time()
            global_step += 1
        tip = 'epoch %d done, cost time:%.2f' % (epoch, time.time() - stime)
        epoch_bar.set_description_str(tip)
        tip_info(tip,out=False)

    return loss_list, auc_list

In [66]:
loss = gloss.SigmoidBinaryCrossEntropyLoss()
net = Model(user_count, item_count, cate_count, embed_size=64, hidden_size=128, attention_size=64, ctx=ctx)
net.initialize(init=init.Xavier(),force_reinit=True, ctx=ctx)

# test

In [67]:
for i, batch in enumerate(test_iter):
    uid, hist_item, hist_cate, hist_item_neg, hist_cate_neg,item,cate, label, sl = [data.as_in_context(ctx) for data in batch[:-1]]
    b = net(uid, hist_item, hist_cate, hist_item_neg, hist_cate_neg, item, cate, sl)   
    print('score',b)
#     print('loss',a)
    break

score 
[ 0.08676144  0.04514597  0.04434888  0.05680565  0.02888979 -0.00232536
  0.13141474 -0.08247334  0.38988367  0.31690297  0.11657902  0.19059235
  0.05171814  0.02063869  0.19938983 -0.0685005  -0.10468584  0.2358349
  0.14914741 -0.02646282 -0.04567137  0.5741412   0.2892901   0.20713036
  0.01690616  0.05846286 -0.20863356  0.07982939  0.17581278 -0.00541545
  0.09921096 -0.03071056  0.13370815  0.28222004  0.25328451 -0.04899491
  0.16147175  0.08039463  0.28692159 -0.06616701  0.05925862  0.04246673
  0.28147456 -0.03420046 -0.04096619  0.2921901   0.15486799 -0.20532647
  0.17587635 -0.11961944 -0.10238911  0.27010387  0.54318792  0.1637041
  0.10268231 -0.08182988  0.26392391  0.07880452 -0.07099348  0.0938931
  0.04157631  0.13418497  0.14684059  0.05490597 -0.10827526  0.16860972
 -0.1135467   0.11178055  0.02927077  0.21986072 -0.08179523  0.12169919
 -0.15752591  0.26792324  0.08669492  0.04968623 -0.09148367  0.17287087
  0.21661702 -0.12213755  0.08942667  0.1800577

In [ ]:
train(net, train_iter, test_iter, loss, train_batch_size, 0.1, 5, ctx, './logs', 'loss_dien','auc_dien')

auc: 0.4977


In [26]:
net

Model(
  (uid_embedding): Embedding(1053 -> 64, float32)
  (item_embedding): Embedding(63001 -> 64, float32)
  (cate_embedding): Embedding(801 -> 64, float32)
  (rnn1): GRU(128 -> 128.0, TNC)
  (rnn2): GRU(None -> 128.0, TNC)
  (attention_layer): Attention(
    (model): Sequential(
      (0): Dense(256 -> 64, Activation(tanh))
      (1): Dense(64 -> 1, linear)
    )
  )
  (mlp): Sequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=576)
    (1): Dense(576 -> 200, Activation(relu))
    (2): Dense(200 -> 80, Activation(relu))
    (3): Dense(80 -> 1, linear)
  )
)